In [1]:
from sifce import datatools, sims, network, tv
import pandas as pd
import numpy as np
from bilby.core.prior import Uniform, Sine, Cosine, PowerLaw, Constraint, DeltaFunction
from pycbc.detector import Detector
from bilby.core.prior import PriorDict
from pycbc.psd import analytical
from astropy.cosmology import Planck18
pd.set_option('display.max_columns', None)

In [2]:
net = network.Network(ifo_uptime_file="segment_list.segs",ifos=["H1", "L1", "V1"], psd_config_file="net_psds.psds")
net.load_psds()
net.psds_for_time(3)

{'H1': <pycbc.types.frequencyseries.FrequencySeries at 0x7f1b32876620>,
 'L1': <pycbc.types.frequencyseries.FrequencySeries at 0x7f1b3284b0a0>,
 'V1': <pycbc.types.frequencyseries.FrequencySeries at 0x7f17ae959720>}

In [3]:
prior_gw150914 = dict(mass_1= DeltaFunction(35),
                     mass_2= DeltaFunction(35),
                     a_1 = DeltaFunction(0.5),
                     a_2 = DeltaFunction(0.5),
                     tilt_1 = DeltaFunction(1.4),
                     tilt_2 = DeltaFunction(1.4),
                     phi_12 = DeltaFunction(3.09),
                     phi_jl = DeltaFunction(5.21),
                     theta_jn =  Sine(name='theta_jn'),
                     phase = DeltaFunction(1.04),
                     reference_frequency = DeltaFunction(20),
                     dec=Cosine(name='dec'),
                     ra=Uniform(name='ra', minimum=0, maximum=2 * np.pi, boundary='periodic'),
                     psi =  Uniform(name='psi', minimum=0, maximum=np.pi, boundary='periodic'),
                     geocent_time = DeltaFunction(5),
                     luminosity_distance = PowerLaw(alpha=2, name='luminosity_distance', minimum=50, maximum=2000, unit='Mpc', latex_label='$d_L$'))

In [4]:
prior_obj = PriorDict(dictionary = prior_gw150914)
pop = sims.SimulationSet(distribution=prior_obj, cosmology_model=Planck18, network=net)
pop.sample_distribution(10)

In [5]:
list_rows_sky = list()
for i in range(len(pop.simulations_dataframe.index)):
    row = pop.simulations_dataframe.iloc[[0]]
    list_rows_sky.append(pop.calc_snrs_sky(row, N=10, benchmark=True))

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
pop.simulations_dataframe = pd.concat(list_rows_sky, ignore_index=True)

In [ ]:
pop.simulations_dataframe

In [ ]:
list_rows_distance = list()
for i in range(len(pop.simulations_dataframe.index)):
    row = pop.simulations_dataframe.iloc[[0]]
    list_rows_distance.append(pop.sample_distances(row, N=1))
    